# Advanced/optional features of `do_datasciencing`

The function `do_datasciencing` has several optional parameters with which you can customize the training. We will explore them in detail in this tutorial.

Note: We will now import `datasciencefunctions` as `ds` and in the rest of this tutorial we will sometimes refer to `datasciencefunctions` as `ds` when discussing the modules of the package and their functions

In [0]:
import datasciencefunctions as ds
import datasciencefunctions.classification as dsclass

#0. Load and prepare data

#### 
* We will load the adult databricks example dataset. 
* It contains categorical, ordinal and numeric (continuous) predictors representing demographic info of US adults and a target denoting whether their income exceeded USD 50 000. 
* We want to train a model to predict whether the person's income exceeds USD 50 000. 
(you can read more details in the readme below).

In [0]:
with open("/dbfs/databricks-datasets/adult/README.md") as f:
    x = ''.join(f.readlines())

print(x)

=============================================================
Adult Dataset
=============================================================
Predict whether income exceeds $50K/yr based on census data. Also known as "Census Income" dataset.

## Source

Donor: 
Ronny Kohavi and Barry Becker 
Data Mining and Visualization 
Silicon Graphics. 
e-mail: ronnyk '@' live.com for questions.

## Data Set Information:
Extraction was done by Barry Becker from the 1994 Census database. A set of reasonably clean records was extracted using the following conditions: ((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0)) 

Prediction task is to determine whether a person makes over 50K a year.

## Attribute Information

Label:
	- >50K, <=50K

Attributes:

	- age: continuous
	- workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked
	- fnlwgt: continuous
	- education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool
	- education-num: continuous
	- marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse
	- occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces
	- relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried
	- race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black
	- sex: Female, Male
	- capital-gain: continuous
	- capital-loss: continuous
	- hours-per-week: continuous
	- native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands

## Citation
If you publish material based on databases obtained from this repository, then, in your acknowledgements, please note the assistance you received by using this repository. This will help others to obtain the same data sets and replicate your experiments. We suggest the following pseudo-APA reference format for referring to this repository:

Lichman, M. (2013). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

In [0]:
schema = """
  age DOUBLE,
  workclass STRING,
  fnlwgt DOUBLE,
  education STRING,
  education_num DOUBLE,
  marital_status STRING,
  occupation STRING,
  relationship STRING,
  race STRING,
  sex STRING,
  capital_gain DOUBLE,
  capital_loss DOUBLE,
  hours_per_week DOUBLE,
  native_country STRING,
  income STRING
"""

df_adult = (
    spark
    .read
    .format("csv")
    .schema(schema)
    .option("header", True)
    .option("path", "dbfs:/databricks-datasets/adult/adult.data")
    .load()
    .sample(fraction=0.35) # only take a sample of the dataset for tutorial purposes
)

df_adult.printSchema()

root
-- age: double (nullable = true)
-- workclass: string (nullable = true)
-- fnlwgt: double (nullable = true)
-- education: string (nullable = true)
-- education_num: double (nullable = true)
-- marital_status: string (nullable = true)
-- occupation: string (nullable = true)
-- relationship: string (nullable = true)
-- race: string (nullable = true)
-- sex: string (nullable = true)
-- capital_gain: double (nullable = true)
-- capital_loss: double (nullable = true)
-- hours_per_week: double (nullable = true)
-- native_country: string (nullable = true)
-- income: string (nullable = true)

When running binary classification, datasciencefunctions expects you to specify a label (or target) column with values 1 and 0.

We will create a classification target column called "income_above_50K" with value 1 if the person's income exceeds USD 50K and 0 otherwise.

In [0]:
import pyspark.sql.functions as F

df_adult_ml = (
    df_adult
    .withColumn(
        "income_above_50K", 
        F.when(F.col("income")==" >50K", 1).otherwise(0)
    )
    # we drop the income column because it is perfectly correlated with our label
    .drop("income")
)

In [0]:
display(df_adult_ml.limit(10))

age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income_above_50K
50.0,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,0
38.0,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,0
28.0,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,0
52.0,Self-emp-not-inc,209642.0,HS-grad,9.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,45.0,United-States,1
31.0,Private,45781.0,Masters,14.0,Never-married,Prof-specialty,Not-in-family,White,Female,14084.0,0.0,50.0,United-States,1
32.0,Private,186824.0,HS-grad,9.0,Never-married,Machine-op-inspct,Unmarried,White,Male,0.0,0.0,40.0,United-States,0
43.0,Self-emp-not-inc,292175.0,Masters,14.0,Divorced,Exec-managerial,Unmarried,White,Female,0.0,0.0,45.0,United-States,1
35.0,Federal-gov,76845.0,9th,5.0,Married-civ-spouse,Farming-fishing,Husband,Black,Male,0.0,0.0,40.0,United-States,0
54.0,?,180211.0,Some-college,10.0,Married-civ-spouse,?,Husband,Asian-Pac-Islander,Male,0.0,0.0,60.0,South,1
49.0,Private,193366.0,HS-grad,9.0,Married-civ-spouse,Craft-repair,Husband,White,Male,0.0,0.0,40.0,United-States,0


## 1. Using MLFlow with `do_datasciencing`
The most straightforward optional parameter of `do_datasciencing` is `use_mlflow`. If it is set to `True` (which it is by default), it tells `do_datasciencing` to log the model summmary (including the model itself) to an MLFlow experiment. 

If you use Databricks and do not specify the MLFlow experient it will be created automatically and linked to the source notebook. However a better practice is to specify where you want your experiment logged. To do that, you just need to import mlflow and set an experiment by denoting a path where it will be stored (if you're using Databricks it is a path within the Databricks Workspace and has to start with `/`)

In [0]:
import mlflow
from datasciencefunctions.utils import current_dbx_notebook_path

# set experiment based on current notebook path, you will probably want to change it to the shared experiment you'll be working on
mlflow.set_experiment(current_dbx_notebook_path(dbutils) + "_test_experiment")

INFO: '/Users/oldrich.vlasic@datasentics.com/DS_lib_CICD_sandbox/classification_chapters/02_advanced_do_datasciencing_test_experiment' does not exist. Creating a new experiment

In [0]:
df_test, df_train, model_summary = (
    dsclass.do_datasciencing(
        df_adult_ml,
        model_type=ds.MlModel.spark_GLM_binomial,
        label_col="income_above_50K",
        # we are logging the model into an MLFlow experiment
        # (you don't actually need to set it to True explicitly as it is True by default)
        use_mlflow=True,
        params_fit_model={
            "max_evals": 2,  # just to make it faster
        },
    )
)

/databricks/spark/python/pyspark/sql/context.py:77: DeprecationWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
 DeprecationWarning)
 0%| | 0/2 [00:00<?, ?trial/s, best loss=?] /databricks/spark/python/pyspark/ml/util.py:758: UserWarning: MLflow auto-tracking of MLlib tuning is currently disabled. To enable this feature, set the flag via `spark.conf.set('spark.databricks.mlflow.trackMLlib.enabled', 'true')` where `spark` is your SparkSession.
 warnings.warn(_MLflowInstrumentation._MLFLOW_TRACKING_DISABLED_MESSAGE)

 0%| | 0/2 [00:00<?, ?trial/s, best loss=?] 50%|█████ | 1/2 [00:29<00:29, 30.00s/trial, best loss: -0.8832280937478466]100%|██████████| 2/2 [00:51<00:00, 27.32s/trial, best loss: -0.8952907680992155]100%|██████████| 2/2 [00:51<00:00, 25.54s/trial, best loss: -0.8952907680992155]

## 2. Skip some columns

Sometimes you want to keep some columns in a dataframe but exclude it from the model features (or label). Typically this would be something like a unique ID of a datapoint. The optional parameter `skip_cols` allows you to do just that, you just need to pass a list of names of columns which will not be considered as features/label during training/prediction.

## 3. Transformation pipeline options - specify categorical and numerical predictors and scaling

By default, `do_datasciencing` uses a heuristic to find categorical and numerical columns automatically based on the number of unique values in each column. However, this is usually only useful for a quick exploration and typically you want to specify your categorical and numerical features manually. Likewise, by default, the features are not scaled in the transformation pipeline and you might want that sometimes.

You can do both in `do_datasciencing` with the use of the optional parameter called `params_fit_pipeline`. This paramater (like other parameters of `do_datasciencing` which start with `params_`) is expected to be a dictionary. It passes specific values to the function `fit_transformation_pipeline` which is one of the functions that do all the work under the hood of `do_datasciencing`. Therefore, you can use the names of the parameters of `fit_transformation_pipeline` which you want to change as keys of the `params_fit_pipeline` dictionary.

**Note:** If you use `params_fit_pipeline` to specify your categorical and/or numerical columns, the heuristic used to find categorical and numerical columns is disabled automatically and all columns not listed among your categorical or numerical columns are ignored by the model (as if listed in the `skip_cols` parameter)

In [0]:
help(ds.data_processing.fit_transformation_pipeline)

Help on function fit_transformation_pipeline in module datasciencefunctions._data_processing:

fit_transformation_pipeline(train_data, cat_cols: list = None, num_cols: list = None, skip_cols: list = None, scaling: str = None)
 Constructs and fits transformation pipeline on training dataset.
 
 In case of Spark, prepares encoder, vector assembler and (optionally)
 scaler on a given dataframe. Scaling type (including no scaling) is
 determined by :param scaling. Equivalently for sklearn.
 
 :param train_data: training data
 :type train_data: PySpark dataframe, Pandas dataframe or list with the Pandas
 dataframe as the first element
 :param cat_cols: a list of column names of categorical features
 :param num_cols: a list of column names of numeric features
 :param skip_cols: names of columns to be excluded from the pipeline
 :param scaling: scaling type to be used: "noscale" (or None), "standard",
 "minmax" for Spark; sklearn uses StandardScaler currently
 :return: fitted pipeline object

From the docstring above (or the [documentation](../../docs)) we see that `cat_cols` and `num_cols` as well as `scaling` are parameters of `fit_transformation_pipeline` and so we can specify their values in the optional parameter `params_fit_pipeline`

In [0]:
cat_list = [
    "workclass",
    "education",
    "marital_status",
    "race",
    "sex",
    "native_country",
]

num_list = [
    "age",
    "capital_gain",
    "capital_loss",
    "hours_per_week",
]

In [0]:
df_test, df_train, model_summary = (
    dsclass.do_datasciencing(
        df_adult_ml,
        model_type=ds.MlModel.spark_GLM_binomial,
        label_col="income_above_50K",
        # we are logging the model as an MLFlow experiment
        use_mlflow=True,
        # we specify the categorical and continuous columns and scaling used
        params_fit_pipeline={
            "cat_cols": cat_list,
            "num_cols": num_list,
            "scaling": "standard",
        },
        params_fit_model={
            "max_evals": 5,  # just to make it faster
        },
    )
)

/databricks/spark/python/pyspark/sql/context.py:77: DeprecationWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
 DeprecationWarning)
 0%| | 0/5 [00:00<?, ?trial/s, best loss=?] /databricks/spark/python/pyspark/ml/util.py:758: UserWarning: MLflow auto-tracking of MLlib tuning is currently disabled. To enable this feature, set the flag via `spark.conf.set('spark.databricks.mlflow.trackMLlib.enabled', 'true')` where `spark` is your SparkSession.
 warnings.warn(_MLflowInstrumentation._MLFLOW_TRACKING_DISABLED_MESSAGE)

 0%| | 0/5 [00:00<?, ?trial/s, best loss=?] 20%|██ | 1/5 [00:13<00:55, 13.89s/trial, best loss: -0.8795936659877259] 40%|████ | 2/5 [00:27<00:41, 13.67s/trial, best loss: -0.8844160533998539] 60%|██████ | 3/5 [00:38<00:26, 13.15s/trial, best loss: -0.8844160533998539] 80%|████████ | 4/5 [00:53<00:13, 13.58s/trial, best loss: -0.8844160533998539]100%|██████████| 5/5 [01:09<00:00, 14.24s/trial, best loss: -0.8844160533998539]100%|██████████| 5/5 [01:09<00:00, 13.87s/trial, best loss: -0.8844160533998539]

## 3. Model fitting options - specify the hyperparameter space and how it is explored

When passing an instance of MlModel to `do_datasciencing`, a default hyperparameter search space is also passed with it. In the default setting `do_datasciencing` uses the [hyperopt](http://hyperopt.github.io/hyperopt/) package to search the default hyperparameter space and select the best hyperparameters based on cross-validation (using the area uder ROC curve as the loss function). Using the optional parameter `params_fit_model` this can be modified in several ways. 

Just like `params_fit_pipeline`, the parameter `params_fit_model` is expected to be a dictionary of parameters of a function used within `do_datasciencing`, in this case it is the `ds.classification.fit_classification_model` function. These values can be modified using `params_fit_model`:


1. `param_space_search` ... can be either set to "hyperopt" (the default value) or "param_grid" for a grid search of the hyperparameter space
2. `validation_technique` ... currently the only option is cross-validation, no point in changing (nothing else will work)
3. `max_evals` ... a parameter relevant when using the hyperopt search, changes the number of hyperopt iterations. Typically, more evaluations lead to better choice of hyperparameters but training time increases with the number of iterations. The default value is set to `30` which might be too low or too high depending on the model, the hyperparameter space and the data.
4. `custom_params` allows you to change the default hyperparameter search space for each model. The format depends on whether you use "param_grid" or "hyperopt" as your space search. Since this is somewhat more complex than the other optional parameters, we will discuss `custom_params` in the last chapter of the tutorial where we also show how tu set up a custom hyperparameter space within the datasciencefunctions framework even without the use of `do_datasciencing`.

In [0]:
df_test, df_train, model_summary = (
    dsclass.do_datasciencing(
        df_adult_ml,
        model_type=ds.MlModel.spark_GLM_binomial,
        label_col="income_above_50K",
        # we are logging the model as an MLFlow experiment
        use_mlflow=True,
        # we specify the categorical and continuous columns and scaling used
        params_fit_pipeline={
            "cat_cols": cat_list,
            "num_cols": num_list,
            "scaling": "standard",
        },
        params_fit_model={
            "max_evals": 10,
        },
    )
)

0%| | 0/10 [00:00<?, ?trial/s, best loss=?] 10%|█ | 1/10 [00:12<01:49, 12.12s/trial, best loss: -0.884895048020474] 20%|██ | 2/10 [00:24<01:37, 12.16s/trial, best loss: -0.8869679422811294] 30%|███ | 3/10 [00:37<01:26, 12.42s/trial, best loss: -0.8869679422811294] 40%|████ | 4/10 [00:49<01:13, 12.33s/trial, best loss: -0.8925697246651214] 50%|█████ | 5/10 [01:02<01:02, 12.54s/trial, best loss: -0.8925697246651214] 60%|██████ | 6/10 [01:15<00:50, 12.57s/trial, best loss: -0.8925697246651214] 70%|███████ | 7/10 [01:29<00:39, 13.10s/trial, best loss: -0.8925697246651214] 80%|████████ | 8/10 [01:42<00:26, 13.15s/trial, best loss: -0.8925697246651214] 90%|█████████ | 9/10 [01:56<00:13, 13.33s/trial, best loss: -0.8925697246651214]100%|██████████| 10/10 [02:10<00:00, 13.47s/trial, best loss: -0.8925697246651214]100%|██████████| 10/10 [02:10<00:00, 13.03s/trial, best loss: -0.8925697246651214]

## 4. Modify the train/test split
If you want to change the default value of the random train/test split ration, fix the randomness of the split via a seed or change some other parameter of the `ds.data_processing.train_test_split` function, you can use the `params_split` optional parameter to change these in the same way as `params_fit_pipeline`.

## 5. Change the parameters of the automatic categorical columns finder

If you want to use the `ds.data_exploration.get_categorical_and_numeric_cols` heuristic to get the categorical and continuous predictors automatically but want to change some of its parameters, you can use `params_get_cat_num_columns` in the same way as `params_fit_pipeline` to pass specific parameter values to `get_categorical_and_numeric_cols`

#FAQ

# Where to now?

[Back to the introductory notebook](classification.ipynb)

[To the next chapter](03_custom_training_pipelines.ipynb)